In [53]:
import requests
import pandas as pd
from sqlalchemy import create_engine


f = open('twitch_secrets.txt')
secret_data = f.read()
f.close()
secret_data


url = "https://api.twitch.tv/helix/streams"

querystring = {"game_id":"509670","first":"100","language":"en"}

Client_ID = secret_data.split('\n')[0]
Client_Secret = secret_data.split('\n')[1]
Oauth_Token = secret_data.split('\n')[2]
PGE_Secret = secret_data.split('\n')[3]

headers = {
    'Client-ID' : Client_ID,
    'Authorization' : 'Bearer ' + Oauth_Token
    }

response = requests.request("GET", url, headers=headers, params=querystring)

#print(response.text)

In [54]:
json_streams = response.json()
json_df = pd.DataFrame(json_streams['data'])

In [55]:
json_df['date_scraped'] = pd.to_datetime('now')
stream_df = json_df.drop(['thumbnail_url','title'], axis=1)

In [56]:
#INPUT YOUR OWN CONNECTION STRING HERE
conn_string = 'postgresql://postgres:'+ PGE_Secret +'@192.168.1.21:5432/postgres'

#perform to_sql test and print result
db = create_engine(conn_string)
conn = db.connect()

stream_df.to_sql('twitch_api_scrape', con=conn, if_exists='append', index=False)

#close connection
conn.close()